# 1. dummy time_embedding 

In [ ]:
import torch 


# (batch_size, tembedding_channels)
temb = torch.randn(2, 512)
temb

tensor([[-1.0657, -0.2787, -1.2110,  ...,  0.5981, -0.5768, -0.5393],
        [ 1.1917, -0.1065,  1.9694,  ..., -0.1435, -1.1355, -0.3291]])

In [1]:
(1,) * 6

(1, 1, 1, 1, 1, 1)

# 2. AdaGroupNorm

`for - images`

In [17]:
from diffusers.models.normalization import AdaGroupNorm
import torch 

# in_channels = 3
out_channels = 6
temb_channels = 6

x = torch.randn(2, out_channels,  64, 64)
temb = torch.randn(2, temb_channels)




ada_group_normalization = AdaGroupNorm(embedding_dim=temb_channels,   # make sure in `embedding_dim` to put the `time_embedding` that is 256 in here.
                                       out_dim=out_channels,
                                       num_groups=2,
                                       eps=1e-6)

print(ada_group_normalization)

ada_group_normalization = ada_group_normalization(x, temb)
ada_group_normalization.shape

AdaGroupNorm(
  (linear): Linear(in_features=6, out_features=12, bias=True)
)


torch.Size([2, 6, 64, 64])

`for - video`

In [2]:
from diffusers.models.normalization import AdaGroupNorm
import torch 

in_channels = 3
out_channels = 64
temb_channels = 8
batch_size = 2
frame = 8 

batch_frame = batch_size * frame

# video_x = torch.randn(batch_size, out_channels, frame,  64, 64)
image_x = torch.randn(batch_frame, out_channels, 64, 64) # so the total image = 2 * 8 => 16 images
temb = torch.randn(batch_frame, temb_channels)




ada_group_normalization = AdaGroupNorm(embedding_dim=temb_channels,   # make sure in `embedding_dim` to put the `time_embedding` that is 256 in here.
                                       out_dim=out_channels,
                                       num_groups=batch_size,
                                       eps=1e-6)

print(ada_group_normalization)

ada_group_normalization = ada_group_normalization(image_x, temb)
ada_group_normalization.shape

AdaGroupNorm(
  (linear): Linear(in_features=8, out_features=128, bias=True)
)
what is the shape of row data: torch.Size([16, 64, 64, 64]) and what is the shape of scale: torch.Size([16, 64, 1, 1]) and what is the shape of shift: torch.Size([16, 64, 1, 1])


torch.Size([16, 64, 64, 64])

# 3. SpatialNorm

In [ ]:
import torch 
from torch import nn 
from torch.nn import functional as F 

class SpatialNorm(nn.Module):
    """
    Spatially conditioned normalization as defined in https://arxiv.org/abs/2209.09002.

    Args:
        f_channels (`int`):
            The number of channels for input to group normalization layer, and output of the spatial norm layer.
        zq_channels (`int`):
            The number of channels for the quantized vector as described in the paper.
    """

    def __init__(
        self,
        f_channels: int,
        zq_channels: int,
    ):
        super().__init__()
        self.norm_layer = nn.GroupNorm(num_channels=f_channels, num_groups=32, eps=1e-6, affine=True)
        self.conv_y = nn.Conv2d(zq_channels, f_channels, kernel_size=1, stride=1, padding=0)
        self.conv_b = nn.Conv2d(zq_channels, f_channels, kernel_size=1, stride=1, padding=0)

    def forward(self, f: torch.Tensor, zq: torch.Tensor) -> torch.Tensor:
        f_size = f.shape[-2:] 
        zq = F.interpolate(zq, size=f_size, mode="nearest")
        norm_f = self.norm_layer(f)
        new_f = norm_f * self.conv_y(zq) + self.conv_b(zq)
        return new_f

In [4]:



num_channels = 64
zq_channels = 8

spatial_norm = SpatialNorm(f_channels=128,
                           zq_channels=16)

print(spatial_norm)

# feature map 
x = torch.randn(2, 128, 64, 64)
# quantized vector 
z = torch.randn(2, 16, 8, 8)

spatial_norm = spatial_norm(x, z)
spatial_norm.shape





SpatialNorm(
  (norm_layer): GroupNorm(32, 128, eps=1e-06, affine=True)
  (conv_y): Conv2d(16, 128, kernel_size=(1, 1), stride=(1, 1))
  (conv_b): Conv2d(16, 128, kernel_size=(1, 1), stride=(1, 1))
)
what is the shape of f : torch.Size([2, 128, 64, 64]) and zq: torch.Size([2, 16, 8, 8])


torch.Size([2, 128, 64, 64])

# 4. torch.interpolate()

#### `mode: 'nearest'`

In [16]:
import torch
import torch.nn.functional as F

# x = torch.rand(1, 1, 2, 2)
x = torch.rand(32, 3, 256, 256)

nearest_interpolate = F.interpolate(x, 
                                #  size=(64, 64),
                                scale_factor=(4, 4),
                                 mode='nearest')
nearest_interpolate.shape


torch.Size([32, 3, 1024, 1024])

#### `mode: 'Linear'`

In [39]:
# batch_size = 32, channels=3, length=4
x_linear = torch.randn(32, 3, 4)




linear_interpolate = F.interpolate(x_linear,
                                   size=8,
                                   mode='linear',
                                   align_corners=False  # a/c to pytorch library
                                   )

linear_interpolate.shape



torch.Size([32, 3, 8])

#### `mode: 'BiLinear'`

In [43]:

bilinear_interpolate = F.interpolate(x,
                                     size=(64, 64),
                                     mode='bilinear')
bilinear_interpolate.shape

torch.Size([32, 3, 64, 64])

# 5. conv 3d

In [3]:
import torch 
from torch import nn 
from typing import Optional, Tuple, Union

class Neural_Network(nn.Module):

    def __init__(self,
                 in_channels: int,
                 out_channels: int,
                 kernel_size: Union[int, Tuple[int, int, int]] = 3,
                 stride: Union[int, Tuple[int, int, int]] = 1,
                 group: Optional[int] = 32,
                 dilation: Optional[int] = 1,
                 padding: Optional[int] = 1):
        
        super().__init__()
        
        self.norm1 = nn.GroupNorm(num_groups=group,
                             num_channels=in_channels,
                             eps=1e-6)
        
        self.conv1 = nn.Conv3d(in_channels=in_channels,
                          out_channels=out_channels,
                          kernel_size=kernel_size,
                          stride=stride,
                          padding=padding,
                          dilation=dilation)
        


    def forward(self, x):

        x = self.norm1(x)
        x = self.conv1(x)
        return x 
    

        


x = torch.randn(2, 16, 8, 256, 256)

neural_network = Neural_Network(in_channels=16,
                                out_channels=16,
                                group=2)

print(neural_network)

output = neural_network(x)
output.shape


Neural_Network(
  (norm1): GroupNorm(2, 16, eps=1e-06, affine=True)
  (conv1): Conv3d(16, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
)


torch.Size([2, 16, 8, 256, 256])

# 6. if add the two different dimension

In [12]:
import torch 

x = torch.randn(2, 16, 32, 32)
temb = torch.rand(2, 16)

temb = temb[:, :, None, None]



add = x + temb
add.shape  # [2, 16, 32, 32]

torch.Size([2, 16, 32, 32])

# 7. torch.chunk()

In [16]:
import torch 

temb = torch.randn(2, 256)

a, b = torch.chunk(temb,
                         chunks=2,
                         dim=1)

a.shape, b.shape



(torch.Size([2, 128]), torch.Size([2, 128]))

In [5]:
import torch 

x = torch.randn(2, 32, 8, 68, 68)

conv = torch.nn.Conv3d(in_channels=32,
                       out_channels=64,
                       kernel_size=3,
                       stride=1,
                       padding=1,
                        dilation=1,
                        groups=2)

conv = conv(x)
conv.shape

torch.Size([2, 64, 8, 68, 68])

# 8. what is the meaning Pixelshuffle() and PixelUnshuffle()

In [ ]:
import torch 


x = torch.randn(2, 4, 256, 256) # output = [2, 1, 512, 512]
x = torch.randn(2, 8, 256, 256) # output = [2, 2, 512, 512]

# make sure the `input_channels` is square of `upscale_factor` 
# out_channels = in_channels / upscaling^2, height, width * upscaling  
pixel_shuffle = torch.nn.PixelShuffle(2) # upscale the channels 
output = pixel_shuffle(x)
output.shape


torch.Size([2, 2, 512, 512])

in_channels = 4 
upscaling = 2*2 = 4 

out_channels = in_channels / upscaling
out_channels = 4 / 4 = 1 


In [2]:
import torch 

x = torch.randn(2, 4, 256, 256)

# out_channels = in_channels * downscale^2, height, width / downscale 
pixel_unshuffle = torch.nn.PixelUnshuffle(2)
output = pixel_unshuffle(x)
output.shape # [2, 1, 128, 128]

torch.Size([2, 16, 128, 128])

In [1]:
block_output = (64,)
block_output[0]

64

# 9. torch.layout()

In [2]:
import torch 




tensor = torch.randn(2, 3)
tensor.layout

converted_shape = tensor.t().stride()
converted_shape

(1, 3)

# 10. torch.generator()



In [12]:
device = torch.device("cuda") if torch.cuda.is_available() else "cpu"

generator = torch.Generator(device=device)
generator.seed()

8784880626046803224

# 11. diffusers.register_to_config()

In [5]:
from diffusers.configuration_utils import register_to_config, ConfigMixin

class vae_Config(ConfigMixin):

    @register_to_config
    def __init__(self,
                 model_name="bert",
                 hidden_size=768,
                 num_layers=12,
                 temporary_param=None,
                 _private_param=None):
        
        self.model_name = model_name
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.temporary_param = temporary_param
        self._private_param = _private_param

In [ ]:
vae_Config.config_name = "vae_Config"
model_config = vae_Config()
model_config

vae_Config {
  "_class_name": "vae_Config",
  "_diffusers_version": "0.33.1",
  "_private_param": null,
  "hidden_size": 768,
  "model_name": "bert",
  "num_layers": 12,
  "temporary_param": null
}

## 12. nn.register_buffer()

In [12]:
import torch
from torch import nn 

class Test(nn.Module):
    def __init__(self):
        super().__init__()
        self.register_buffer('tensor1', torch.tensor([0.2, 0.3, 0.4])[None, :, None, None])
        

    def forward(self):
        return self.tensor2
    

a = Test()
a = a()
a.shape

torch.Size([1, 3, 1, 1])

## 13. torchvision.models()

In [20]:
from torchvision import models

vgg_16 = models.vgg16().features
vgg_16

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (17): Conv2d(256, 512, kernel_si

In [50]:
import torch
from torch import nn 
from collections import namedtuple
from torchvision import models


class Vgg16(nn.Module):
    def __init__(self,
                 requires_grad=False,
                 pretrained=True):
        super().__init__()
        
        vgg_pretrained_features = models.vgg16().features
        self.slice1 = nn.Sequential()
        self.slice2 = nn.Sequential()
        self.slice3 = nn.Sequential()
        self.slice4 = nn.Sequential()
        self.slice5 = nn.Sequential()
        
        
        for i in range(4): 
            self.slice1.add_module(name=str(i), module=vgg_pretrained_features[i]) 
        for i in range(4, 9): 
            self.slice2.add_module(name=str(i), module=vgg_pretrained_features[i])
        for i in range(9, 16):  
            self.slice3.add_module(name=str(i), module=vgg_pretrained_features[i])
        for i in range(16, 23):  
            self.slice4.add_module(name=str(i), module=vgg_pretrained_features[i])
        for i in range(23, 30):  
            self.slice5.add_module(name=str(i), module=vgg_pretrained_features[i])

        if not requires_grad:
            for param in self.parameters():
                param.requires_grad = False


    def forward(self, x):
        h1 = self.slice1(x)
        h2 = self.slice2(h1)
        h3 = self.slice3(h2)
        h4 = self.slice4(h3)
        h5 = self.slice5(h4)

        vgg_outputs = namedtuple(typename="VggOutputs", field_names=['h1', 'h2', 'h3', 'h4', 'h5'])
        out = vgg_outputs(h1, h2, h3, h4, h5)

        return out 
    

    

out = Vgg16()
x = torch.randn(2, 3, 256, 256)
out = out(x)
out.h1.shape

torch.Size([2, 64, 256, 256])

In [ ]:
class NetLinLayer(nn.Module):
    """A single linear layer which does a 1x1 conv."""

    def __init__(self,
                 in_channels,
                 out_channels: int = 1,
                 use_dropout: bool = False):
        super().__init__()
        self.model = nn.Sequential([
            nn.Dropout(),
            nn.Conv2d(in_channels=in_channels,
                      out_channels=out_channels,
                      kernel_size=1,
                      stride=1,
                      padding=0,
                      bias=False)
        ])

out = NetLinLayer(in_channels=3)
out

In [21]:
import torch

print(torch.ones(())) # zero dim 
print(torch.ones(1)) # one dim 
print(torch.ones(2, 2)) # two dim 
print(torch.ones(2, 2, 2))  # three dim
print(torch.ones(2, 3, 128, 128))   # four dim
print(torch.ones(2, 3, 4, 128, 128))   # five dim



tensor(1.)
tensor([1.])
tensor([[1., 1.],
        [1., 1.]])
tensor([[[1., 1.],
         [1., 1.]],

        [[1., 1.],
         [1., 1.]]])
tensor([[[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]],

         [[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]],

         [[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]]],


        [[[1., 1., 1.,  ..., 1., 1., 1.],
        

## 15. init distributed mode

Of course. This Python code is designed to set up a distributed computing environment, specifically for situations where the script is launched using Open MPI (a common framework for parallel computing). It acts as a compatibility layer, translating Open MPI's environment variables into a format that a different library, likely PyTorch Distributed, expects.

However, there appears to be a **critical bug** in the code you provided.

-----

### \#\# Code Explanation

The primary goal is to detect if the program is running in a multi-process, multi-machine setup managed by Open MPI and then configure it accordingly.

1.  **Detecting the Open MPI Environment**

    ```python
    if int(os.getenv('OMPI_COMM_WORLD_SIZE', '0')) > 0:
    ```

      * This line checks for an environment variable named `OMPI_COMM_WORLD_SIZE`.
      * When you launch a program with Open MPI's `mpirun`, it automatically sets this variable to the total number of processes you requested. For example, `mpirun -np 8 ...` would set it to `'8'`.
      * `os.getenv(..., '0')` safely gets the value, defaulting to `'0'` if it doesn't exist.
      * The `if` condition is true only if the script is launched by Open MPI with more than one process.

2.  **Reading Open MPI Variables**

    ```python
    rank = int(os.environ['OMPI_COMM_WORLD_RANK'])
    local_rank = int(os.environ['OMPI_COMM_WORLD_LOCAL_RANK'])
    world_size = int(os.environ['OMPI_COMM_WORLD_SIZE'])
    ```

    If the code is in an Open MPI environment, it reads these key variables:

      * `OMPI_COMM_WORLD_SIZE`: The **world size**, or the total number of processes in the job.
      * `OMPI_COMM_WORLD_RANK`: The **global rank**, a unique ID for the current process, from `0` to `world_size - 1`.
      * `OMPI_COMM_WORLD_LOCAL_RANK`: The **local rank**, which is the process's unique ID *on its specific machine*. This is crucial for assigning one GPU per process on a multi-GPU server.

3.  **Mapping to Standard Variables (with a Bug)**

    ```python
    # This block contains an error
    os.environ["LOCAL_RANK"] = os.environ['OMPI_COMM_WORLD_LOCAL_RANK']
    os.environ["RANK"] = os.environ['OMPI_COMM_WORLD_SIZE']
    os.environ["WORLD_SIZE"] = os.environ['OMPI_COMM_WORLD_RANK']
    ```

    This section tries to create the standard environment variables (`LOCAL_RANK`, `RANK`, `WORLD_SIZE`) that frameworks like PyTorch Distributed use for auto-configuration.

      * `LOCAL_RANK` is set correctly.
      * **BUG:** `RANK` and `WORLD_SIZE` are swapped. The global rank is being set to the world size, and the world size is being set to the global rank. This will cause the distributed setup to fail.

4.  **Setting Program Arguments**

    ```python
    args.rank = int(os.environ["RANK"])
    args.world_size = int(os.environ["WORLD_SIZE"])
    args.gpu = int(os.environ["LOCAL_RANK"])
    ```

    Finally, it populates an `args` object (likely from `argparse`) with these values so the rest of the application can easily use them, for instance, to assign the correct GPU to the process.

-----

### \#\# The Corrected Code

To fix the bug, you need to swap the assignments for `RANK` and `WORLD_SIZE`.

```python
# Corrected version
if int(os.getenv('OMPI_COMM_WORLD_SIZE', '0')) > 0:

    # Set the standard variables correctly
    os.environ["LOCAL_RANK"] = os.environ['OMPI_COMM_WORLD_LOCAL_RANK']
    os.environ["RANK"] = os.environ['OMPI_COMM_WORLD_RANK']          # Corrected
    os.environ["WORLD_SIZE"] = os.environ['OMPI_COMM_WORLD_SIZE']      # Corrected

    # Populate the args object from the newly set variables
    args.rank = int(os.environ["RANK"])
    args.world_size = int(os.environ["WORLD_SIZE"])
    args.gpu = int(os.environ["LOCAL_RANK"])
```

In short, this code is a bridge that allows a program expecting PyTorch's distributed environment to be launched using Open MPI's `mpirun` command.

In [ ]:
import os 
import argparse as args

if int(os.getenv('OMPI_COMM_WORLD_SIZE', '0')) > 0:
    rank = int(os.environ['OMPI_COMM_WORLD_RANK'])
    local_rank = int(os.environ['OMPI_COMM_WORLD_LOCAL_RANK'])
    world_size = int(os.environ['OMPI_COMM_WORLD_SIZE'])

    os.environ["LOCAL_RANK"] = os.environ['OMPI_COMM_WORLD_LOCAL_RANK']
    os.environ["RANK"] = os.environ['OMPI_COMM_WORLD_RANK']
    os.environ["WORLD_SIZE"] = os.environ['OMPI_COMM_WORLD_SIZE']

    args.rank = int(os.environ["RANK"])
    args.world_size = int(os.environ["WORLD_SIZE"])
    args.gpu = int(os.environ["LOCAL_RANK"])

elif 'RANK' in os.environ and 'WORLD_SIZE' in os.environ:
    args.rank = int(os.environ["RANK"])
    args.world_size = int(os.environ['WORLD_SIZE'])
    args.gpu = int(os.environ['LOCAL_RANK'])

else:
    print('Not using distributed mode')
    args.distributed = False

    


This code configures the settings for a distributed training session, most commonly used in a framework like PyTorch. Each line sets a specific parameter for how multiple processes (running on different GPUs or machines) should coordinate.

---

### ## `args.distributed = True`

This is a simple boolean flag that **enables distributed mode**. 🚩

Your training script will likely have `if` statements that check this flag. If it's `True`, the script will execute code to set up communication between processes. If `False`, it will run as a standard, single-process program.

---

### ## `args.dist_backend = 'nccl'`

This line sets the **communication backend** to `'nccl'`.

The backend is the underlying library that manages how data (like gradients and model weights) is sent between your GPUs.

* **`nccl`** stands for the **NVIDIA Collective Communications Library**. It is a highly optimized library for communication between NVIDIA GPUs. It's the standard and fastest choice for multi-GPU training on NVIDIA hardware. Other options exist, like `gloo` (for CPUs) or `mpi`, but `nccl` is preferred for deep learning with CUDA.

---

### ## `args.dist_url = "env://"`

This line tells the program **how to find the other processes** to form a communication group.

The `"env://"` method instructs the program to initialize itself by reading connection details from the **environment variables**. This is the most common and flexible approach because a launch utility like PyTorch's `torchrun` will automatically set the required variables (`MASTER_ADDR`, `MASTER_PORT`, `RANK`, and `WORLD_SIZE`) for each process it starts. This way, you don't have to hardcode IP addresses or other machine-specific details into your script.

In summary, these three lines work together to tell your script: "Yes, run in **distributed mode** using the fast **NCCL backend** for GPUs, and get the connection details from the **environment variables** set up by the launcher."

In [ ]:
args.distributed = True
args.dist_backend = 'nccl'
args.dist_url = "env://"

This code block initializes the PyTorch Distributed Data Parallel (DDP) environment, which is the standard way to perform multi-GPU or multi-machine training in PyTorch. It ensures that all the separate processes can communicate and synchronize with each other.

-----

### \#\# Conditional Initialization

```python
if init_pytorch_ddp:
```

This is a simple check. The code inside this `if` block only runs if the `init_pytorch_ddp` variable is `True`. This is a useful way to control whether the script should handle the DDP setup itself or if it's being managed by an external framework like Hugging Face's `accelerate` (as the comment suggests).

-----

### \#\# Assigning a GPU

```python
torch.cuda.set_device(args.gpu)
```

This line is crucial for multi-GPU setups. It **binds the current process to a specific GPU**. 📌

In distributed training, you launch one process for each GPU. This command uses the `args.gpu` value (which is typically the process's **local rank**) to ensure that each process exclusively controls one GPU. For example, the process with `local_rank=0` takes control of GPU 0, `local_rank=1` takes GPU 1, and so on. This prevents conflicts and ensures memory is managed correctly.

-----

### \#\# Initializing the Process Group

```python
torch.distributed.init_process_group(backend=args.dist_backend,
                                     init_method=args.dist_url,
                                     world_size=args.world_size,
                                     rank=args.rank,
                                     timeout=timedelta(days=365))
```

This is the main command that **establishes the connection between all processes**. 🤝 It's like the official start of a conference call where everyone joins and can now hear each other.

  * `backend=args.dist_backend`: Sets the communication library, which is typically `'nccl'` for fast communication between NVIDIA GPUs.
  * `init_method=args.dist_url`: Tells the processes how to find each other. As discussed, this is usually `"env://"` so the processes can read the connection info from environment variables.
  * `world_size=args.world_size`: Informs this process about the total number of processes participating in the training job.
  * `rank=args.rank`: Gives this specific process its unique global ID.
  * `timeout=timedelta(days=365)`: This sets an extremely long timeout for operations. If one process has to wait for another for longer than this period, it will raise an error. Setting a long timeout can prevent your job from crashing during long data loading phases or when debugging.

-----

### \#\# Synchronizing All Processes

```python
torch.distributed.barrier()
```

This command acts as a **synchronization point**. 🚧

When a process executes `barrier()`, it will pause and wait until **every single process** in the group has also reached this exact point. This ensures that no process moves on to the next step (like loading the model) until all other processes have successfully finished the initialization. It prevents race conditions and guarantees everyone starts on the same page.

In [ ]:
from datetime import timedelta
init_pytorch_ddp = True

if init_pytorch_ddp:
    # Init DDP Group, for script without using accelerate framework
    torch.cuda.set_device(args.gpu)
    torch.distributed.init_process_group(backend=args.dist_backend, 
                                         init_method=args.dist_url,
                                        world_size=args.world_size, 
                                        rank=args.rank, 
                                        timeout=timedelta(days=365))
    torch.distributed.barrier()

This function cleverly modifies Python's built-in `print()` function to control which process can write output in a distributed computing environment.

Its main purpose is to **prevent log spam**. In distributed training, you run the same script on multiple GPUs or machines simultaneously. If every process prints logs, your console output becomes a messy and unreadable wall of duplicated text. This function ensures that, by default, **only the main process (the "master") can print messages**.

-----

### \#\# How It Works

1.  **Saves the Original `print`**:

    ```python
    import builtins as __builtin__
    builtin_print = __builtin__.print
    ```

    First, it gets a reference to the original, built-in `print` function and saves it in a variable called `builtin_print`. This is essential because it needs a way to call the real `print` function later.

2.  **Creates a New, Custom `print`**:

    ```python
    def print(*args, **kwargs):
        force = kwargs.pop('force', False)
        if is_master or force:
            builtin_print(*args, **kwargs)
    ```

    Next, it defines a new function, which is also named `print`. This new function contains the special logic:

      * It checks for a special keyword argument called `force`. If you call `print("message", force=True)`, it will set `force` to `True`. Otherwise, `force` is `False`.
      * The `if is_master or force:` condition is the core of the function. It allows printing only if one of two things is true:
          * The process is the master process (`is_master` is `True`).
          * The print call was explicitly told to override the rule (`force` is `True`).
      * If the condition passes, it calls the original `builtin_print` to display the message. Otherwise, it does nothing, effectively silencing the print statement.

3.  **Replaces the Built-in `print`**:

    ```python
    __builtin__.print = print
    ```

    Finally, this line overwrites the global, built-in `print` with the new custom `print` function. This technique is called **monkey-patching**. From this point on, whenever any part of your code calls `print()`, it will execute your new, conditional version instead of the original one.

-----

### \#\# Practical Example

Imagine you call this function at the start of your script:

```python
# In the master process (rank 0)
is_master = True
setup_for_distributed(is_master)

# In a worker process (rank > 0)
is_master = False
setup_for_distributed(is_master)

# --- Now, in both processes ---
print("Epoch 1 starting...")  # Only prints on the master process
print("Debugging worker.", force=True) # Prints on ALL processes because of 'force'
```

In [ ]:
def setup_for_distributed(is_master):
    """
    This function disables printing when not in master process
    """
    import builtins as __builtin__
    builtin_print = __builtin__.print

    def print(*args, **kwargs):
        force = kwargs.pop('force', False)
        
        if force==False:
            raise "make sure `force=True`"
        
        if is_master or force:
            builtin_print(*args, **kwargs)

    __builtin__.print = print

---

In [1]:
import torch
import torch.nn as nn

class SimpleModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(10, 20)
        self.activation = nn.ReLU()
        self.layer2 = nn.Linear(20, 5)

model = SimpleModel()

# Print the name of each learnable parameter
for name, param in model.named_parameters():
    print(f"Name: {name}, Shape: {param.shape}")

# --- Output ---
# Name: layer1.weight, Shape: torch.Size([20, 10])
# Name: layer1.bias, Shape: torch.Size([20])
# Name: layer2.weight, Shape: torch.Size([5, 20])
# Name: layer2.bias, Shape: torch.Size([5])

Name: layer1.weight, Shape: torch.Size([20, 10])
Name: layer1.bias, Shape: torch.Size([20])
Name: layer2.weight, Shape: torch.Size([5, 20])
Name: layer2.bias, Shape: torch.Size([5])


In [1]:
car = {
  "brand": "Ford",
  "model": "Mustang",
  "year": 1964
}

x = car.get('model', [])

print(x)

Mustang


In [5]:
optimizer = 'adamW'
optimizer.lower()

'adamw'

In [13]:
fruits = {'applebananache': 'fav'}

fruits.pop('applebananacherry', [])

[]

In [17]:
skip = {}
skip_list = None
if skip_list is not None:
    skip = skip_list
print(skip)

{}


In [ ]:
iterable = 'hello worl how are you an this the understanding' 
a = str(len(iterable))  # 48
b = str(len(a)) # 2
b

'2'

In [5]:
import torch 

x =torch.arange(1., 6.)
x = torch.topk(x, k=2)
x

torch.return_types.topk(
values=tensor([5., 4.]),
indices=tensor([4, 3]))

In [1]:
print("hello\t world")

hello	 world


In [1]:
int(0.1 * 4)

0